In [1]:
import time
import numpy as np

from sklearn.base import BaseEstimator
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression as LR
from sklearn.svm import LinearSVC as LSVC
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:

"""This file shows a couple of implementations of the perceptron learning
algorithm. It is based on the code from Lecture 3, but using the slightly
more compact perceptron formulation that we saw in Lecture 6.

There are two versions: Perceptron, which uses normal NumPy vectors and
matrices, and SparsePerceptron, which uses sparse vectors and matrices.
The latter may be faster when we have high-dimensional feature representations
with a lot of zeros, such as when we are using a "bag of words" representation
of documents.
"""

class LinearClassifier(BaseEstimator):
    """
    General class for binary linear classifiers. Implements the predict
    function, which is the same for all binary linear classifiers. There are
    also two utility functions.
    """

    def decision_function(self, X):
        """
        Computes the decision function for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """
        return X.dot(self.w)

    def predict(self, X):
        """
        Predicts the outputs for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """

        # First compute the output scores
        scores = self.decision_function(X)

        # Select the positive or negative class label, depending on whether
        # the score was positive or negative.
        out = np.select([scores >= 0.0, scores < 0.0],
                        [self.positive_class,
                         self.negative_class])
        return out

    def find_classes(self, Y):
        """
        Finds the set of output classes in the output part Y of the training set.
        If there are exactly two classes, one of them is associated to positive
        classifier scores, the other one to negative scores. If the number of
        classes is not 2, an error is raised.
        """
        classes = sorted(set(Y))
        if len(classes) != 2:
            raise Exception("this does not seem to be a 2-class problem")
        self.positive_class = classes[1]
        self.negative_class = classes[0]

    def encode_outputs(self, Y):
        """
        A helper function that converts all outputs to +1 or -1.
        """
        return np.array([1 if y == self.positive_class else -1 for y in Y])


In [3]:
class Perceptron(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        # Perceptron algorithm:
        for i in range(self.n_iter):
            for x, y in zip(X, Ye):

                # Compute the output score for this instance.
                score = x.dot(self.w)

                # If there was an error, update the weights.
                if y*score <= 0:
                    self.w += y*x


In [4]:
##### The following part is for the optional task.

### Sparse and dense vectors don't collaborate very well in NumPy/SciPy.
### Here are two utility functions that help us carry out some vector
### operations that we'll need.

def add_sparse_to_dense(x, w, factor):
    """
    Adds a sparse vector x, scaled by some factor, to a dense vector.
    This can be seen as the equivalent of w += factor * x when x is a dense
    vector.
    """
    w[x.indices] += factor * x.data

def sparse_dense_dot(x, w):
    """
    Computes the dot product between a sparse vector x and a dense vector w.
    """
    return np.dot(w[x.indices], x.data)


class SparsePerceptron(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm,
    assuming that the input feature matrix X is sparse.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.

        Note that this will only work if X is a sparse matrix, such as the
        output of a scikit-learn vectorizer.
        """
        self.find_classes(Y)

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        Ye = self.encode_outputs(Y)

        # Initialize the weight vector to all zeros.
        self.w = np.zeros(X.shape[1])

        # Iteration through sparse matrices can be a bit slow, so we first
        # prepare this list to speed up iteration.
        XY = list(zip(X, Ye))

        for i in range(self.n_iter):
            for x, y in XY:

                # Compute the output score for this instance.
                # (This corresponds to score = x.dot(self.w) above.)
                score = sparse_dense_dot(x, self.w)

                # If there was an error, update the weights.
                if y*score <= 0:
                    # (This corresponds to self.w += y*x above.)
                    add_sparse_to_dense(x, self.w, y)


In [5]:
class LinearSVC(LinearClassifier):
    """
    An implementation of the PEGASOS algorithm for linear support vector classifier
    """

    def __init__(self, n_iter=10, weight=1, normalize=False, verbose=False):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set. Floats allowed. E.g. 1.5 or 0.5 times
        Weight the value of the size of w in the object function to minimize
        Normalize W on each iteration to stay within the ball of radius 1/sqrt(weight)
        """
        self.n_iter = n_iter
        self.weight = weight
        self.normalize = normalize
        self.verbose = verbose

    def fit(self, X, Y):
        """
        Train a linear classifier using the PEGASOS learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)

        if self.verbose:
            print("Epoch | Objective | Loss\n-------------------------")

        # Pegasos algorithm:
        for e in range(self.n_iter): # e=epochs
            for t in range(1, n_samples + 1):
                #Draw random sample from training data
                i = np.random.randint(0, n_samples)
                y, x = Ye[i], X[i]

                #calculate step-size
                eta = 1/(self.weight*(t + e*n_samples)) #weight=lambda
                #predict for this sample
                score = x.dot(self.w)
                #adjust w
                self.w *= 1-eta*self.weight
                #If we predict wrongly or with to small margin
                if y*score < 1:
                    #adjust w
                    self.w += eta*y*x

                #optional
                if self.normalize:
                    #limit w to ball with radius 1/sqrt(weight)
                    self.w *= min(1, (1/self.weight**0.5)/np.linalg.norm(self.w))
            
            if self.verbose:
                loss = np.sum(np.maximum(0, 1 - Ye*X.dot(self.w)))
                objective = self.weight/2*np.linalg.norm(self.w)**2 + loss/n_samples
                print(f"  {e}  |  {objective:.2f}  |  {loss:.2f}")                    

In [6]:
class LogisticRegression(LinearClassifier):
    """
    Logistic Regression classifier
    """

    def __init__(self, n_iter=10, weight=1, normalize=False, verbose=False):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set. Floats allowed. E.g. 1.5 or 0.5 times
        Weight the value of the size of w in the object function to minimize
        Normalize W on each iteration to stay within the ball of radius 1/sqrt(weight)
        """
        self.n_iter = n_iter
        self.weight = weight
        self.normalize = normalize
        self.verbose = verbose



    def fit(self, X, Y):
        """
        Pegasos logistic regression
        """
        self.find_classes(Y)
        Ye = self.encode_outputs(Y)
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)

        if self.verbose:
            print("Epoch | Objective | Loss\n-------------------------")

        for e in range(self.n_iter):
            for j in range(1, n_samples + 1):
                #Draw random sample from training data
                i = np.random.randint(0, n_samples)
                y, x = Ye[i], X[i]

                t = j + e * n_samples # j will loop 1,..,n_samples, we want t monotonously increasingly 

                #calculate step-size
                eta = 1/(self.weight*t)
                #predict for this sample
                score = x.dot(self.w)
                #adjust w
                self.w = (1-eta*self.weight)*self.w + eta*y/(1+np.exp(y*score))*x

                #optional
                if self.normalize:
                    #limit w to ball with radius 1/sqrt(weight)
                    self.w *= min(1, (1/self.weight**0.5)/np.linalg.norm(self.w))

            if self.verbose:
                loss = np.sum(np.log(1+np.exp(-Ye * X.dot(self.w))))
                objective = self.weight/2*np.linalg.norm(self.w)**2 + loss/n_samples
                print(f"  {e}  |  {objective:.2f}  |  {loss:.2f}")

In [7]:
# This function reads the corpus, returns a list of documents, and a list
# of their corresponding polarity labels. 
def read_data(corpus_file):
    X = []
    Y = []
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            _, y, _, x = line.split(maxsplit=3)
            X.append(x.strip())
            Y.append(y)
    return X, Y


In [8]:
# Read all the documents.
X, Y = read_data('data/all_sentiment_shuffled.txt')

# Split into training and test parts.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                random_state=0)

In [9]:
def pipeliner(trainingAlgorithms, kbest = 1000):
    if not isinstance(trainingAlgorithms, list): #wrap as list if not list
        trainingAlgorithms = [trainingAlgorithms]
        
    pipelines = []
    for t in trainingAlgorithms:
        p = make_pipeline(TfidfVectorizer(), Normalizer(), t)
        # p = make_pipeline(TfidfVectorizer(), SelectKBest(k=kbest), Normalizer(), t)
        pipelines.append((p.steps[-1][0], p))
    
    return pipelines

In [10]:
pipes = pipeliner([Perceptron(), SparsePerceptron(),
                    LinearSVC(n_iter=10, weight=1/len(Xtrain), normalize=True, verbose=True),
                    LogisticRegression(n_iter=10, weight=1/len(Xtrain), normalize=True, verbose=True), 
                    LSVC(), LR()])

In [11]:
for p in pipes: #the last two pipes are scikit learn implementation, for comparison
    name, pipe = p
    t0 = time.time()
    pipe.fit(Xtrain, Ytrain)
    t1 = time.time()
    print(f"{name} Training time: {t1-t0:.2f} sec.")
    # Evaluate on the test set.
    Yguess = pipe.predict(Xtest)
    print(f"{name} Accuracy: {accuracy_score(Ytest, Yguess):.4f}.")

perceptron Training time: 6.71 sec.
perceptron Accuracy: 0.8300.
sparseperceptron Training time: 2.16 sec.
sparseperceptron Accuracy: 0.8300.
Epoch | Objective | Loss
-------------------------
  0  |  0.49  |  2645.97
  1  |  0.39  |  2244.11
  2  |  0.37  |  2089.19
  3  |  0.36  |  2024.68
  4  |  0.35  |  1939.55
  5  |  0.34  |  1873.36
  6  |  0.33  |  1839.08
  7  |  0.33  |  1817.39
  8  |  0.33  |  1801.25
  9  |  0.33  |  1778.44
linearsvc Training time: 15.96 sec.
linearsvc Accuracy: 0.8414.
Epoch | Objective | Loss
-------------------------
  0  |  0.48  |  3338.68
  1  |  0.45  |  3297.84
  2  |  0.45  |  3282.09
  3  |  0.44  |  3270.26
  4  |  0.44  |  3264.59
  5  |  0.44  |  3264.43
  6  |  0.44  |  3269.34
  7  |  0.44  |  3263.86
  8  |  0.44  |  3262.86
  9  |  0.44  |  3264.86
logisticregression Training time: 19.20 sec.
logisticregression Accuracy: 0.8296.
linearsvc Training time: 0.91 sec.
linearsvc Accuracy: 0.8410.
logisticregression Training time: 1.49 sec.
log

### Bonus task 1 - Improve computation times of linear svc

#### a) BLAS

In [12]:
import scipy.linalg.blas as blas

class BlasLinearSVC(LinearClassifier):
    """
    An implementation of the PEGASOS algorithm for linear support vector classifier
    """

    def __init__(self, n_iter=10, weight=1, normalize=False, verbose=False):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set. Floats allowed. E.g. 1.5 or 0.5 times
        Weight the value of the size of w in the object function to minimize
        Normalize W on each iteration to stay within the ball of radius 1/sqrt(weight)
        """
        self.n_iter = n_iter
        self.weight = weight
        self.normalize = normalize
        self.verbose = verbose

    def fit(self, X, Y):
        """
        Train a linear classifier using the PEGASOS learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)

        if self.verbose:
            print("Epoch | Objective | Loss\n-------------------------")

        # Pegasos algorithm:
        for e in range(self.n_iter): # e=epochs
            for t in range(1, n_samples + 1):
                #Draw random sample from training data
                i = np.random.randint(0, n_samples)
                y, x = Ye[i], X[i]

                #calculate step-size
                eta = 1/(self.weight*(t + e*n_samples)) #weight=lambda
                #predict for this sample
                score = blas.ddot(x, self.w)
                #adjust w
                blas.dscal(1-eta*self.weight, self.w)
                
                #If we predict wrongly or with to small margin
                if y*score < 1:
                    #adjust w
                    blas.daxpy(x, self.w, a=eta*y)
                    # self.w += eta*y*x
                
                #optional
                if self.normalize:
                    #limit w to ball with radius 1/sqrt(weight)
                    # self.w *= min(1, (1/self.weight**0.5)/np.linalg.norm(self.w))
                    blas.dscal(min(1, 1/(blas.ddot(self.w, self.w)*self.weight)**0.5), self.w)
            
            if self.verbose:
                loss = np.sum(np.maximum(0, 1 - Ye*X.dot(self.w)))
                objective = self.weight/2*blas.ddot(self.w, self.w) + loss/n_samples
                print(f"  {e}  |  {objective:.2f}  |  {loss:.2f}")                    


#### b) using sparse vectors

In [14]:
class SparseLinearSVC(LinearClassifier):
    """
    An implementation of the PEGASOS algorithm for linear support vector classifier
    """

    def __init__(self, n_iter=10, weight=1, normalize=False, verbose=False):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set. Floats allowed. E.g. 1.5 or 0.5 times
        Weight the value of the size of w in the object function to minimize
        Normalize W on each iteration to stay within the ball of radius 1/sqrt(weight)
        """
        self.n_iter = n_iter
        self.weight = weight
        self.normalize = normalize
        self.verbose = verbose

    def fit(self, X, Y):
        """
        Train a linear classifier using the PEGASOS learning algorithm. X sparse
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)


        # Initialize the weight vector to all zeros.
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)

        if self.verbose:
            print("Epoch | Objective | Loss\n-------------------------")

        # Pegasos algorithm:
        for e in range(self.n_iter): # e=epochs
            for t in range(1, n_samples + 1):
                #Draw random sample from training data
                i = np.random.randint(0, n_samples)
                y, x = Ye[i], X[i]

                #calculate step-size
                eta = 1/(self.weight*(t + e*n_samples)) #weight=lambda
                #predict for this sample
                score = sparse_dense_dot(x, self.w)
                #adjust w
                self.w *= 1-eta*self.weight
                #If we predict wrongly or with to small margin
                if y*score < 1:
                    #adjust w
                    add_sparse_to_dense(x, self.w, eta*y)
                    # self.w = self.w + eta*y*x
                 

#### c) sped up scaling of sparse solution

In [24]:
class QuickSparseLinearSVC(LinearClassifier):
    """
    An implementation of the PEGASOS algorithm for linear support vector classifier
    """

    def __init__(self, n_iter=10, weight=1, normalize=False, verbose=False):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set. Floats allowed. E.g. 1.5 or 0.5 times
        Weight the value of the size of w in the object function to minimize
        Normalize W on each iteration to stay within the ball of radius 1/sqrt(weight)
        """
        self.n_iter = n_iter
        self.weight = weight
        self.normalize = normalize
        self.verbose = verbose

    def fit(self, X, Y):
        """
        Train a linear classifier using the PEGASOS learning algorithm. X sparse
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # Initialize the weight vector to all zeros.
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)

        if self.verbose:
            print("Epoch | Objective | Loss\n-------------------------")        

        scaling = 1
        # Pegasos algorithm:
        for e in range(self.n_iter): # e=epochs            
            for t in range(2, n_samples + 1):
                #Draw random sample from training data
                i = np.random.randint(0, n_samples)
                y, x = Ye[i], X[i]

                #calculate step-size
                eta = 1/(self.weight*(t + e*n_samples)) #weight=lambda
                #predict for this sample
                score = scaling*sparse_dense_dot(x, self.w)
                
                #If we predict wrongly or with to small margin
                if y*score < 1:
                    #adjust w
                    add_sparse_to_dense(x, self.w, eta*y/scaling)

                scaling *= 1-eta*self.weight                                          

        self.w *= scaling

<div class="alert alert-info alert-block">

#### Results comparison
</div>

In [23]:
name, pipe = pipeliner(LinearSVC(n_iter=10, weight=1/len(Xtrain)), kbest='all')[0]

t0 = time.time()
pipe.fit(Xtrain, Ytrain)
t1 = time.time()
print(f"{name} Training time: {t1-t0:.2f} sec.")
# Evaluate on the test set.
Yguess = pipe.predict(Xtest)
print(f"{name} Accuracy: {accuracy_score(Ytest, Yguess):.4f}.")

name, pipe = pipeliner(BlasLinearSVC(n_iter=10, weight=1/len(Xtrain)), kbest='all')[0]

t0 = time.time()
pipe.fit(Xtrain, Ytrain)
t1 = time.time()
print(f"{name} Training time: {t1-t0:.2f} sec.")
# Evaluate on the test set.
Yguess = pipe.predict(Xtest)
print(f"{name} Accuracy: {accuracy_score(Ytest, Yguess):.4f}.")

name, pipe = pipeliner(SparseLinearSVC(n_iter=10, weight=1/len(Xtrain)), kbest='all')[0]

t0 = time.time()
pipe.fit(Xtrain, Ytrain)
t1 = time.time()
print(f"{name} Training time: {t1-t0:.2f} sec.")
# Evaluate on the test set.
Yguess = pipe.predict(Xtest)
print(f"{name} Accuracy: {accuracy_score(Ytest, Yguess):.4f}.")


name, pipe = pipeliner(QuickSparseLinearSVC(n_iter=10, weight=1/len(Xtrain)), kbest='all')[0]

t0 = time.time()
pipe.fit(Xtrain, Ytrain)
t1 = time.time()
print(f"{name} Training time: {t1-t0:.2f} sec.")
# Evaluate on the test set.
Yguess = pipe.predict(Xtest)
print(f"{name} Accuracy: {accuracy_score(Ytest, Yguess):.4f}.")

linearsvc Training time: 11.81 sec.
linearsvc Accuracy: 0.8368.
blaslinearsvc Training time: 7.83 sec.
blaslinearsvc Accuracy: 0.8439.
sparselinearsvc Training time: 9.61 sec.
sparselinearsvc Accuracy: 0.8431.
quicksparselinearsvc Training time: 7.72 sec.
quicksparselinearsvc Accuracy: 0.8393.
